# Etapa 2 – Transformação de Dados

Este notebook executa a Etapa 2 do teste técnico da IntuitiveCare:

- Extrair dados da tabela do Anexo I (PDF)
- Salvar os dados em formato `.csv`
- Compactar o CSV em um arquivo `.zip`
- Substituir as siglas das colunas OD e AMB por suas descrições completas

In [8]:
from google.colab import drive
import os

# Montar o Google Drive
drive.mount('/content/drive')

# Mudar para a pasta do projeto
os.chdir("/content/drive/MyDrive/intuitivecare_teste_tecnico")
print("Diretório atual:", os.getcwd())

Mounted at /content/drive
Diretório atual: /content/drive/MyDrive/intuitivecare_teste_tecnico


In [14]:
!pip install pymupdf pandas
!pip install pandas
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 87.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 79.0 MB/s eta 0:00:00


Extração dos dados com pdfplumber

In [17]:
import pdfplumber
import pandas as pd

pdf_path = "content/web_scraping/anexo_I.pdf"
todas_linhas = []

with pdfplumber.open(pdf_path) as pdf:
    for pagina in pdf.pages:
        tabelas = pagina.extract_tables()
        for tabela in tabelas:
            for linha in tabela:
                if any(campo and campo.strip() for campo in linha):
                    todas_linhas.append(linha)

dados = pd.DataFrame(todas_linhas)

Renomeando colunas

In [18]:
dados.columns = dados.iloc[0]
dados = dados.drop(index=0).reset_index(drop=True)

Limpeza de linhas vazias ou cabeçalho repetido

In [19]:
dados = dados.dropna(how='all')

dados = dados[dados[dados.columns[0]] != dados.columns[0]]
dados = dados.reset_index(drop=True)

Data frame - estrutura

In [20]:
print("Formato:", dados.shape)
dados.info()
display(dados.head())

Formato: (3392, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3392 entries, 0 to 3391
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PROCEDIMENTO    3392 non-null   object
 1   RN
(alteração)  3392 non-null   object
 2   VIGÊNCIA        3392 non-null   object
 3   OD              3392 non-null   object
 4   AMB             3392 non-null   object
 5   HCO             3392 non-null   object
 6   HSO             3392 non-null   object
 7   REF             3392 non-null   object
 8   PAC             3392 non-null   object
 9   DUT             3392 non-null   object
 10  SUBGRUPO        3392 non-null   object
 11  GRUPO           3392 non-null   object
 12  CAPÍTULO        3392 non-null   object
dtypes: object(13)
memory usage: 344.6+ KB


,PROCEDIMENTO,RN\n(alteração),VIGÊNCIA,OD,AMB,HCO,HSO,REF,PAC,DUT,SUBGRUPO,GRUPO,CAPÍTULO
0,ACONSELHAMENTO GENÉTICO,,,,AMB,HCO,HSO,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
1,ATENDIMENTO/ACOMPANHAMENTO EM HOSPITAL-DIA PSI...,,,,,HCO,HSO,REF,,109,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
2,ATENDIMENTO INTEGRAL AO RECÉM-NASCIDO (SALA DE...,,,,,HCO,,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
3,ATENDIMENTO MÉDICO DO INTENSIVISTA EM UTI GERA...,,,,,HCO,HSO,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS
4,ATENDIMENTO PEDIÁTRICO A GESTANTES (3º TRIMESTRE),,,,AMB,HCO,,REF,,,"CONSULTAS, VISITAS HOSPITALARES OU\nACOMPANHAM...",PROCEDIMENTOS GERAIS,PROCEDIMENTOS GERAIS


Salvar como CSV e compactar em .zip

In [21]:
csv_path = "content/transformacao/Teste_Letycia.csv"
zip_path = "content/transformacao/Teste_Letycia.zip"

dados.to_csv(csv_path, index=False)

import zipfile

with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write(csv_path, arcname="Teste_Letycia.csv")

print("CSV e ZIP gerados com sucesso.")

CSV e ZIP gerados com sucesso.


Substituir siglas OD e AMB pela descrição

In [22]:
# Substituições (se essas colunas existirem)
if "OD" in dados.columns:
    dados["OD"] = dados["OD"].replace("X", "Consulta médica em consultório ou ambulatório")

if "AMB" in dados.columns:
    dados["AMB"] = dados["AMB"].replace("X", "Procedimento realizado em ambiente ambulatorial")

##Etapa 2 finalizada